# Ансамбли

### OzonMasters, "Машинное обучение 1"

В этом ноутбуке вам предлагается реализовать алгоритмы бустинга и бэггинга.

In [1]:
import numpy as np
import numpy.testing as npt
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.datasets import make_classification

## 1. Сэмплирование случайных объектов и признаков

Во многих ансамблевых алгоритмах используется прием, заключающийся в обучении на случайной подвыборке объектов или на случайном подмножестве признаков.

Так что для начала реализуем класс, который будет упрощать семплирование различных подмассивов данных

В классе `ObjectSampler` надо реализовать метод `sample`, который возвращает случайную подвыборку объектов обучения и ответы для них

In [14]:
class BaseSampler:
    def __init__(self, max_samples=1.0, bootstrap=False):
        """
        Parameters
        ----------
        bootstrap : Boolean
            if True then use bootstrap sampling
        max_samples : float in [0;1]
            proportion of sampled examples
        """
        self.bootstrap = bootstrap
        self.max_samples = max_samples
    
    def sample(self, x):
        raise NotImplementedError

class ObjectSampler(BaseSampler):
    def __init__(self, axis=0, max_samples=1.0, bootstrap=True):
        """
        Parameters
        ----------
        axis : int
            which axis use to sample
        """
        self.axis = axis
        super().__init__(max_samples=max_samples, bootstrap=bootstrap)
    
    def sample(self, x, y):
        """
        Parameters
        ----------
        x : numpy ndarray of shape (n_objects, n_features)
        y : numpy ndarray of shape (n_objects,)
        
        Returns
        -------
        x_sampled, y_sampled : numpy ndarrays of shape (n_samples, n_features) and (n_samples,)
            n_samples = x_sampled.shape[0] * self.max_samples
        """
        idx = np.random.choice(x.shape[0], int(x.shape[0] * self.max_samples), self.bootstrap)
        x_sampled = x[idx]
        y_sampled = y[idx]
        return x_sampled, y_sampled

В классе `FeaturesSampler` надо реализовать метод `sample`, который возвращает случайную подвыборку индексов признаков, по которым будет производится обучение

In [23]:
class FeaturesSampler(BaseSampler):
    def __init__(self, axis=1, max_samples=1.0, bootstrap=False):
        self.axis = axis
        super().__init__(max_samples=max_samples, bootstrap=bootstrap)
        
    def sample(self, x):
        """
        Parameters
        ----------
        x : numpy ndarray of shape (n_objects, n_features)
        
        Returns
        -------
        indices : numpy ndarrays of shape (n_features_sampled)
        """
        
        indices = np.random.choice(x.shape[1], int(x.shape[1] * self.max_samples), self.bootstrap)
        return indices

In [24]:
some_X = np.array([[0, 1, 2], [0.3, 1, 3], [0.5, 1, 3], [1, 2, 1]])
some_y = np.array([1, 5, 3, 1])

object_sampler = ObjectSampler(max_samples=0.7)
feature_sampler = FeaturesSampler(max_samples=0.7)

assert object_sampler.sample(some_X, some_y)[0].shape == (int(0.7*some_X.shape[0]), some_X.shape[1])
assert object_sampler.sample(some_X, some_y)[1].shape == (int(0.7*some_y.shape[0]),)

sample_X, sample_y = object_sampler.sample(some_X, some_y)

for sub_x, sub_y in zip(sample_X, sample_y):
    assert sub_x.tolist() in some_X.tolist()
    assert sub_y in sample_y

assert feature_sampler.sample(some_X).shape == (int(0.7 * some_X.shape[1]),)

## 2. Бэггинг

Суть бэггинга заключается в обучении нескольких 'слабых' базовых моделей и объединении их в одну модель, обладающую бОльшей обобщающей способностью. Каждая базовая модель обучается на случайно выбранном подмножестве объектов и на случайно выбранном подмножестве признаков для этих объектов.

Ниже вам предлагается реализовать несколько методов класса `Bagger`:
* `fit` - обучение базовых моделей
* `predict_proba` - вычисление вероятностей ответов
* `predict` - вычисление ответов

In [37]:
class Bagger:
    def __init__(
        self, base_estimator,
        object_sampler, feature_sampler,
        n_estimators=10, **params
    ):
        """
        n_estimators : int
            number of base estimators
        base_estimator : class
            class for base_estimator with fit(), predict() and predict_proba() methods
        feature_sampler : instance of FeaturesSampler
        object_sampler : instance of ObjectSampler
        estimators : list
            list for containing base_estimator instances
        indices : list
            list for containing feature indices for each estimator
        params : kwargs
            params for base_estimator initialization
        """
        self.n_estimators = n_estimators
        self.base_estimator = base_estimator
        self.feature_sampler = feature_sampler
        self.object_sampler = object_sampler
        self.estimators = []
        self.indices = []
        self.params = params
    
    def fit(self, X, y):
        """
        for i in range(self.n_estimators):
            1) select random indices of features for current estimator
            2) select random objects and answers for train
            3) fit base_estimator (don't forget to remain only selected features)
            4) save base_estimator (self.estimators) and feature indices (self.indices)
        
        NOTE that self.base_estimator is class and you should init it with
        self.base_estimator(**self.params) before fitting
        """
        for i in range(self.n_estimators):
            features = self.feature_sampler.sample(X)
            X_samples, y_samples = self.object_sampler.sample(X, y)
            estimator = self.base_estimator(**self.params)
            estimator.fit(X_samples[:, features], y_samples)
            self.estimators.append(estimator)
            self.indices.append(features)
    
    def predict_proba(self, X):
        """
        Returns
        -------
        probas : numpy ndarrays of shape (n_objects, n_classes)
        
        Calculate mean value of all probas from base_estimators
        Don't forget, that each estimator has its own feature indices for prediction
        """
        probas = self.estimators[0].predict_proba(X[:, self.indices[0]])
        for i in range(1, self.n_estimators):
            probas += self.estimators[i].predict_proba(X[:, self.indices[i]])
        return np.array(probas) / self.n_estimators
    
    def predict(self, X):
        """
        Returns
        -------
        predictions : numpy ndarrays of shape (n_objects, )
        
        """
        predictions = np.argmax(self.predict_proba(X), axis=1)
        return predictions

Для проверки, обучим бэггинг над решающими деревьями (случайный лес)

In [38]:
class RandomForestClassifier(Bagger):
    def __init__(self, n_estimators=30, max_depth=None, min_samples_leaf=1):
        base_estimator = DecisionTreeClassifier
        objects_sampler = ObjectSampler(max_samples=0.9)
        features_sampler = FeaturesSampler(max_samples=0.8)
        
        super().__init__(
            base_estimator=base_estimator,
            object_sampler=object_sampler,
            feature_sampler=feature_sampler,
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_leaf=min_samples_leaf
        )

In [39]:
some_random_forest = RandomForestClassifier()

some_X, some_y = make_classification(n_samples=30, n_features=50,
                                     n_informative=50, n_redundant=0,
                                     random_state=0, shuffle=False)

some_random_forest.fit(some_X, some_y)
predictions = some_random_forest.predict(some_X)
assert isinstance(predictions, type(np.zeros(0)))
npt.assert_equal(predictions, some_y)

## 3. Бустинг

Бустинг последовательно обучает набор базовых моделей таким образом, что каждая следующая модель пытается исправить ошибки работы предыдущей модели. Логика того, как учитываются ошибки предыдущей модели может быть разной. В алгоритме градиентного бустинга каждая следующая модель обучается на "невязках" предыдущей модели, минимизируя итоговую функцию потерь. Причем вклад каждой следующей модели уменьшается с каждой итерацией. В данной реализации мы будем постоянно домножать вклад на коэффициент `lr`.

То есть, при обучении очередной модели, ответ модифицируется следующим образом:
$$y_{updated} = preds_{base} - lr * update,$$
    где $preds_{base}$ - ответ предыдущей модели, $update$ - ответ текущей модели

Ниже вам предлагается реализовать стратегию обучения базовых классификаторов для `GradientBoostingClassifier`:
* `_fit_base_estimator` - обучение базовой модели
* `_gradient` - расчет градиента функции ошибки
В данном случае мы используем логистическую функцию потерь, градиент которой рассчитыватся как:
$$\nabla L(y, pred) = pred - y$$

А также метод `predict` для класса `Booster`. Учитывая, что каждая модель улучшает предсказания предыдущей, то предсказания обновляются как: `preds = preds - lr * estimator[i].predict`

Так как мы работаем с потерями функции потерь для классификации и вероятностями классификации, то метки классов приводятся к one-hot виду. В качестве базовых моделей используются модели регрессии, чтобы предсказывать остаток.

In [51]:
class Booster:
    def __init__(
        self, base_estimator, feature_sampler,
        n_estimators=10, lr=.5, **params
    ):
        """
        n_estimators : int
            number of base estimators
        base_estimator : class
            class for base_estimator with fit(), predict() and predict_proba() methods
        feature_sampler : instance of FeaturesSampler
        estimators : list
            list for containing base_estimator instances
        indices : list
            list for containing feature indices for each estimator
        lr : float
            learning rate for estimators
        params : kwargs
            kwargs for base_estimator init
        """
        self.n_estimators = n_estimators
        self.base_estimator = base_estimator
        self.feature_sampler = feature_sampler
        self.estimators = []
        self.indices = []
        self.lr = lr
        self.params = params
    
    def _to_categorical(self, x, n_col=None):
        """ 
        One-hot encoding of nominal values
        """
        if not n_col:
            n_col = np.amax(x) + 1
        one_hot = np.zeros((x.shape[0], n_col))
        one_hot[np.arange(x.shape[0]), x] = 1
        return one_hot
    
    def _fit_first_estimator(self, X, y):
        """
        fits first model and saves it to self.estimators
        (and its indicies to self.indices)
        """
        feature_indices = self.feature_sampler.sample(X)
        self.indices.append(feature_indices)
        curr_estimator = self.base_estimator(**self.params).fit(np.take(X, feature_indices, axis=1), y)
        self.estimators.append(curr_estimator)
        
    def _fit_base_estimator(self, X, y):
        raise NotImplementedError
    
    def fit(self, X, y):
        """
        iteratively fits base models
        """
        y = self._to_categorical(y)
        self._fit_first_estimator(X, y)
        predictions_base = self.estimators[-1].predict(np.take(X, self.indices[-1], axis=1))
        for i in range(self.n_estimators - 1):
            predictions_base = self._fit_base_estimator(X, y, predictions_base)
    
    def predict_proba(self, X):
        """
        Returns
        -------
        predictions : numpy ndarrays of shape (n_objects, n_classes)
        -------
        1) get predictions by first model (self.estimators[0])
        2) for each estimator in self.estimators[1:] (and its indicies):
            update predictions
        3) turn into probability distribution as
        probas = np.exp(predictions) / np.expand_dims(np.sum(np.exp(predictions), axis=1), axis=1)
        """
        predictions = self.estimators[0].predict(X[:, self.indices[0]])
        for estimator, idx in zip(self.estimators[1:], self.indices[1:]):
            predictions -= self.lr * estimator.predict(X[:, idx])
        return np.exp(predictions) / np.expand_dims(np.sum(np.exp(predictions), axis=1), axis=1)
    
    def predict(self, X):
        probas = self.predict_proba(X)
        return np.argmax(probas, axis=1)

In [52]:

class GradientBoostingClassifier(Booster):
    
    def __init__(self, n_estimators=30, lr=0.5, max_depth=None, min_samples_leaf=1):
        """
        n_estimators : int
            number of base estimators
        base_estimator : class
            class for base_estimator with fit(), predict() and predict_proba() methods
        feature_sampler : instance of FeaturesSampler
        estimators : list
            list for containing base_estimator instances
        indices : list
            list for containing feature indices for each estimator
        lr : float
            learning rate for estimators
        params : kwargs
            kwargs for base_estimator init
        """
        
        base_estimator = DecisionTreeRegressor
        feature_sampler = FeaturesSampler(max_samples=0.8)
        super().__init__(
            base_estimator=base_estimator,
            feature_sampler=feature_sampler,
            n_estimators=n_estimators,
            lr=lr,
            max_depth=max_depth,
            min_samples_leaf=min_samples_leaf
        )
        
    def _gradient(self, y, pred):
        """
        Returns
        -------
        gradient : numpy ndarrays of shape (n_objects, n_classes)
        """
        gradient = pred - y
        return gradient
    
    def _fit_base_estimator(self, X, y, predictions_base):
        """
        X : numpy ndarrays of shape (n_objects, n_features)
        y : numpy ndarrays of shape (n_objects, )
        predictions_base : numpy ndarrays of shape (n_objects, n_classes)
            updated predictions from previous step
        -------
        Returns
        -------
        y_updated : numpy ndarrays of shape (n_objects, n_classes)
            updated predictions
        -------
        
        1) calculate gradient
        2) select random indices of features for current estimator
        3) fit estimator with predictions_base as target
        4) calculate y_updated
        5) save estimator and indicies
        
        NOTE that self.base_estimator is class and you should init it with
        self.base_estimator(**self.params) before fitting
        """
        grad = self._gradient(y, predictions_base)
        features = self.feature_sampler.sample(X)
        estimator = self.base_estimator(**self.params).fit(X[:, features], grad)
        y_updated = predictions_base - self.lr * estimator.predict(X[:, features])
        self.estimators.append(estimator)
        self.indices.append(features)
        return y_updated
    

In [53]:
some_gradient_classifier = GradientBoostingClassifier()
some_gradient_classifier.fit(some_X, some_y)
predictions = some_gradient_classifier.predict(some_X)
assert isinstance(predictions, type(np.zeros(0)))
npt.assert_equal(predictions, some_y)